In [ ]:
# Import required libraries for the chatbot interface
import random  # For randomly selecting responses from available options
import json  # For loading the intents.json file with responses
import torch  # PyTorch library for loading the trained model
from model import NeuralNet  # Import our custom neural network class
from preprocessing import bag_of_words, tokenize  # Import text processing functions

NLTK data downloaded successfully!


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/princemarcelle/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/princemarcelle/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
# Set up device for inference (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Check for GPU availability

# Load the intents data containing responses for each intent category
with open('intents.json', 'r') as f:  # Open the intents.json file
    intents = json.load(f)  # Parse JSON content into Python dictionary

# Load the saved model data from the training phase
FILE = 'data.pth'  # Define the saved model file
data = torch.load(FILE)  # Load the saved data dictionary

# Extract model parameters and training data from saved file
input_size = data['input_size']  # Get the input layer size (vocabulary size)
hidden_size = data['hidden_size']  # Get the hidden layer size
output_size = data['output_size']  # Get the output layer size (number of intent categories)
all_words = data['all_word']  # Get the vocabulary (all unique words)
tags = data['tags']  # Get the list of intent tags
model_state = data['model_state']  # Get the trained model weights and biases

# Recreate the neural network model with the same architecture
model = NeuralNet(input_size, hidden_size, output_size).to(device)  # Create model and move to device
model.load_state_dict(model_state)  # Load the trained weights into the model
model.eval()  # Set model to evaluation mode (no training, only inference)

NeuralNet(
  (l1): Linear(in_features=114, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=13, bias=True)
  (relu): ReLU()
)

In [ ]:
# Set up the chatbot interface
bot_name = "Sam"  # Define the chatbot's name
print("let's chat! Type quit to exit")  # Display welcome message and instructions

# Main chat loop - continues until user types "quit"
while True:
    sentence = input("you: ")  # Get user input
    if sentence == "quit":  # Check if user wants to exit
        break  # Exit the chat loop

    # Process the user's input for prediction
    sentence = tokenize(sentence)  # Split the sentence into individual words/tokens
    X = bag_of_words(sentence, all_words)  # Convert sentence to bag-of-words vector
    X = X.reshape(1, X.shape[0])  # Reshape for batch processing (add batch dimension)
    X = torch.from_numpy(X)  # Convert numpy array to PyTorch tensor

    # Get prediction from the neural network
    output = model(X)  # Forward pass through the model
    _, predicted = torch.max(output, dim=1)  # Get the index of the highest probability class
    tag = tags[predicted.item()]  # Convert index back to intent tag name

    # Calculate confidence score for the prediction
    probs = torch.softmax(output, dim=1)  # Convert raw outputs to probabilities
    prob = probs[0][predicted.item()]  # Get the probability of the predicted class

    # Respond based on confidence level
    if prob.item() > 0.75:  # If confidence is above 75%
        # Find the matching intent and select a random response
        for intent in intents['intents']:  # Loop through all intents
            if tag == intent['tag']:  # Find the matching intent category
                print(f'{bot_name}: {random.choice(intent['responses'])}')  # Print random response
    else:
        # If confidence is too low, indicate lack of understanding
        print(f'{bot_name}: I do not understand...')  # Print fallback response
            
        

let's chat! Type quit to exit


you:  hey


Sam: Hello! Ready to connect with your family?


you:  what price should i get


Sam: We believe family connection should be accessible! Our subscription plans include free features for basic family networking and premium options for advanced features. Check our pricing page for details.


...

you:  how do i create an account


Sam: You can create family events easily! Go to the Events section, click 'Create Event', add details like date, time, location, and description. Family members can RSVP and you'll get real-time updates.


you:  bye


Sam: Take care! Your family is always here waiting for you.


you:  quit
